In [1]:
import pandas as pd

In [2]:
TARGET_COLUMNS = [ 
    'hospital_ccn',
    'hospital_ein',
    'code_type',
    'internal_code',
    'description',
    'code_orig',
    'code_prefix',
    'code',
    'modifier',
    'ndc',
    'ndc_num_hcpcs_doses',
    'ndc_quantity_desc',
    'apc',
    'rev_code',
    'rev_desc',
    'billing_class',
    'patient_class',
    'payer_category',
    'payer_orig',
    'payer_name',
    'plan_orig',
    'plan_name',
    'plan_id',
    'plan_type',
    'rate',
    'rate_method',
    'rate_desc',
    'filename',
    'file_last_updated',
    'url',
    'permalink'
]

In [3]:
ccn = "460041"
url = "https://content.steward.org/sites/default/files/2023-01/680562507_DavisHospital_standardcharges.csv"
filename = url.split("/")[-1]
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
import subprocess
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘680562507_DavisHospital_standardcharges.csv’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://content.steward.org/sites/default/files/2023-01/680562507_DavisHospital_standardcharges.csv', '-O', '680562507_DavisHospital_standardcharges.csv'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, dtype={'Rev Code': str, 'DRG': str}, low_memory=False)
df_in

,As of Date,Line Type,Line ID,Charge Code/Package,Charge Description,DRG,Patient Type,CPT,Modifiers,Rev Code,...,UHC HMO/PPO/POS,UHC Medicare Advantage,United Behavioral Health,United Behavioral Health Medicaid,Universal Health Network,Utah Cancer Control Program,WISE Fully Insured Plan,WISE Standard Network,Workers Comp Fund of Utah (WCFU),Workers Comp Non Par
0,1/20/2023,CDM,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,272,...,446.88,NaN,588.00,588.00,441.00,588.00,317.52,335.16,411.60,499.80
1,1/20/2023,CDM,2,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,47.12,NaN,62.00,62.00,46.50,62.00,33.48,35.34,43.40,52.70
2,1/20/2023,CDM,3,1000273S,SET RADIAL ARTERY 20GA,NaN,OP,NaN,,272,...,25.73,NaN,33.85,33.85,25.39,33.85,18.28,19.29,23.70,28.77
3,1/20/2023,CDM,4,1000506S,X-ELECTRODE PRENDIVILLE LOOP 1,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
4,1/20/2023,CDM,5,1000507S,X-ELECTRODE BALL 3MM,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18075,1/20/2023,DRG,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,NaN,...,55908.40,23128.54,19484.65,19484.65,14613.49,19484.65,10521.71,36070.92,13639.25,16561.95
18076,1/20/2023,DRG,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,NaN,...,28580.95,12345.90,19735.35,19735.35,14801.51,19735.35,10657.09,19687.81,13814.74,16775.05
18077,1/20/2023,DRG,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,NaN,...,18573.24,8397.15,17411.30,17411.30,13058.48,17411.30,9402.10,11706.23,12187.91,14799.61
18078,1/20/2023,DRG,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,NaN,...,NaN,NaN,11377.95,11377.95,8533.46,11377.95,6144.09,6599.21,7964.56,9671.26


In [6]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'Line Type': 'code_type',
    'As of Date': 'file_last_updated',
    'Line ID': 'internal_code',
    'Charge Code/Package': 'code_orig',
    'Charge Description': 'description',
    'DRG': 'drg',
    'CPT': 'cpt',
    'Patient Type': 'patient_class',
    'Modifiers': 'modifier',
    'Rev Code': 'rev_code',
})

df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,...,UHC HMO/PPO/POS,UHC Medicare Advantage,United Behavioral Health,United Behavioral Health Medicaid,Universal Health Network,Utah Cancer Control Program,WISE Fully Insured Plan,WISE Standard Network,Workers Comp Fund of Utah (WCFU),Workers Comp Non Par
0,1/20/2023,CDM,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,272,...,446.88,NaN,588.00,588.00,441.00,588.00,317.52,335.16,411.60,499.80
1,1/20/2023,CDM,2,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,47.12,NaN,62.00,62.00,46.50,62.00,33.48,35.34,43.40,52.70
2,1/20/2023,CDM,3,1000273S,SET RADIAL ARTERY 20GA,NaN,OP,NaN,,272,...,25.73,NaN,33.85,33.85,25.39,33.85,18.28,19.29,23.70,28.77
3,1/20/2023,CDM,4,1000506S,X-ELECTRODE PRENDIVILLE LOOP 1,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
4,1/20/2023,CDM,5,1000507S,X-ELECTRODE BALL 3MM,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18075,1/20/2023,DRG,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,NaN,...,55908.40,23128.54,19484.65,19484.65,14613.49,19484.65,10521.71,36070.92,13639.25,16561.95
18076,1/20/2023,DRG,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,NaN,...,28580.95,12345.90,19735.35,19735.35,14801.51,19735.35,10657.09,19687.81,13814.74,16775.05
18077,1/20/2023,DRG,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,NaN,...,18573.24,8397.15,17411.30,17411.30,13058.48,17411.30,9402.10,11706.23,12187.91,14799.61
18078,1/20/2023,DRG,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,NaN,...,NaN,NaN,11377.95,11377.95,8533.46,11377.95,6144.09,6599.21,7964.56,9671.26


In [7]:
df_mid['code_type'] = 'none'
df_mid.loc[df_mid['cpt'].notnull(), 'code_type'] = 'cpt'
df_mid.loc[df_mid['drg'].notnull(), 'code_type'] = 'drg'
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,...,UHC HMO/PPO/POS,UHC Medicare Advantage,United Behavioral Health,United Behavioral Health Medicaid,Universal Health Network,Utah Cancer Control Program,WISE Fully Insured Plan,WISE Standard Network,Workers Comp Fund of Utah (WCFU),Workers Comp Non Par
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,272,...,446.88,NaN,588.00,588.00,441.00,588.00,317.52,335.16,411.60,499.80
1,1/20/2023,none,2,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,47.12,NaN,62.00,62.00,46.50,62.00,33.48,35.34,43.40,52.70
2,1/20/2023,none,3,1000273S,SET RADIAL ARTERY 20GA,NaN,OP,NaN,,272,...,25.73,NaN,33.85,33.85,25.39,33.85,18.28,19.29,23.70,28.77
3,1/20/2023,none,4,1000506S,X-ELECTRODE PRENDIVILLE LOOP 1,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
4,1/20/2023,none,5,1000507S,X-ELECTRODE BALL 3MM,NaN,OP,NaN,,272,...,353.40,NaN,465.00,465.00,348.75,465.00,251.10,265.05,325.50,395.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18075,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,NaN,...,55908.40,23128.54,19484.65,19484.65,14613.49,19484.65,10521.71,36070.92,13639.25,16561.95
18076,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,NaN,...,28580.95,12345.90,19735.35,19735.35,14801.51,19735.35,10657.09,19687.81,13814.74,16775.05
18077,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,NaN,...,18573.24,8397.15,17411.30,17411.30,13058.48,17411.30,9402.10,11706.23,12187.91,14799.61
18078,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,NaN,...,NaN,NaN,11377.95,11377.95,8533.46,11377.95,6144.09,6599.21,7964.56,9671.26


In [8]:
df_mid['code_type'].value_counts()

cpt     9284
none    7949
drg      847
Name: code_type, dtype: int64

In [9]:
money_columns = df_mid.columns.to_list()[10:]
remaining_columns = df_mid.columns.to_list()[:10]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_orig', value_name='rate')
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,272,Gross Charges,588.00
1,1/20/2023,none,2,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,Gross Charges,62.00
2,1/20/2023,none,3,1000273S,SET RADIAL ARTERY 20GA,NaN,OP,NaN,,272,Gross Charges,33.85
3,1/20/2023,none,4,1000506S,X-ELECTRODE PRENDIVILLE LOOP 1,NaN,OP,NaN,,272,Gross Charges,465.00
4,1/20/2023,none,5,1000507S,X-ELECTRODE BALL 3MM,NaN,OP,NaN,,272,Gross Charges,465.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,NaN,Workers Comp Non Par,16561.95
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,NaN,Workers Comp Non Par,16775.05
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,NaN,Workers Comp Non Par,14799.61
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,NaN,Workers Comp Non Par,9671.26


In [10]:
money_columns

['Gross Charges',
 'Discounted Cash Price',
 'De-Identified Minimum',
 'De-Identified Maximum',
 '50 PCT',
 '75 PCT',
 'Advantage Work Comp Services (AWCS)',
 'Aetna Altius Peak Preference',
 'Aetna AutoLiv',
 'Aetna Legacy Network Commercial',
 'Aetna NAP',
 'Aetna Signature Administrators',
 'Aetna Standard Network',
 'Altius/Coventry/First Health Workers Compensation',
 'American Health Management Services',
 'American Horizon Financial',
 'Asserta Health',
 'BCBS BlueOption',
 'BCBS FOCAL POINT',
 'BCBS Medicare Advantage UT',
 'BCBS SAVEWELL PPO',
 'BCBS Traditional',
 'BCBS ValueCare/PPO',
 'Beech Street',
 'Beech Street/Multiplan Complementary Network',
 'California Medicaid OOS',
 'CARESACT',
 'Cigna Behavioral Health',
 'Cigna Healthsprings',
 'Cigna HMO',
 'Cigna Individual Family HIX',
 'CIGNA LOCAL PLUS',
 'Cigna NBN',
 'Cigna PPO',
 'Clearfield Job Corps',
 'COBA Health',
 'Commercial Non-Contracted',
 'Corvel Workers Comp',
 'Davis Behavioral Health',
 'Disability Determi

In [11]:
df_mid['rev_code'] = df_mid['rev_code'].fillna('na')

def pad_rev_code_if_needed(rev_code):
    if type(rev_code) == str and rev_code != 'na':
        if len(rev_code) == 3:
            return '0' + rev_code
        elif len(rev_code) == 2:
            return '00' + rev_code
        elif len(rev_code) == 1:
            return '000' + rev_code

    return rev_code

df_mid['rev_code'] = df_mid['rev_code'].apply(pad_rev_code_if_needed)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,0272,Gross Charges,588.00
1,1/20/2023,none,2,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,0272,Gross Charges,62.00
2,1/20/2023,none,3,1000273S,SET RADIAL ARTERY 20GA,NaN,OP,NaN,,0272,Gross Charges,33.85
3,1/20/2023,none,4,1000506S,X-ELECTRODE PRENDIVILLE LOOP 1,NaN,OP,NaN,,0272,Gross Charges,465.00
4,1/20/2023,none,5,1000507S,X-ELECTRODE BALL 3MM,NaN,OP,NaN,,0272,Gross Charges,465.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,na,Workers Comp Non Par,16561.95
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,na,Workers Comp Non Par,16775.05
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,na,Workers Comp Non Par,14799.61
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,na,Workers Comp Non Par,9671.26


In [12]:
df_mid_cpt = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'cpt'])
df_mid_cpt['code'] = df_mid['cpt']
df_mid_cpt['code_prefix'] = 'hcpcs_cpt'
df_mid_cpt

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code,code_prefix
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,0272,Gross Charges,588.00,C1751,hcpcs_cpt
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,NaN,OP,A6537,,0272,Gross Charges,11.96,A6537,hcpcs_cpt
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,NaN,OP,C1729,,0272,Gross Charges,75.90,C1729,hcpcs_cpt
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,NaN,OP,C1894,,0272,Gross Charges,230.18,C1894,hcpcs_cpt
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,NaN,IP,A4648,,0278,Gross Charges,31.25,A4648,hcpcs_cpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770963,1/20/2023,cpt,17204,9430000032,PHY/QHP OP PULM RHB W/MNTR,NaN,OP,94626,,0948,Workers Comp Non Par,154.63,94626,hcpcs_cpt
1770964,1/20/2023,cpt,17205,9430000111,PULMONARY STRESS TESTING,NaN,OP,94618,,0460,Workers Comp Non Par,340.20,94618,hcpcs_cpt
1770973,1/20/2023,cpt,17214,97748S,DRESSING OIL EMULSION 3X8 CRTY,NaN,OP,A6233,,0272,Workers Comp Non Par,1.28,A6233,hcpcs_cpt
1770990,1/20/2023,cpt,17231,999556S,CATHETER DCI P04010,NaN,OP,C1877,,0272,Workers Comp Non Par,3652.11,C1877,hcpcs_cpt


In [13]:
df_mid_drg = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'drg'])
df_mid_drg['code'] = df_mid_drg['drg']
df_mid_drg

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code
17233,1/20/2023,drg,17234,MS-DRG 001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1,IP,NaN,NaN,na,Gross Charges,846592.03,1
17234,1/20/2023,drg,17235,MS-DRG 002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,2,IP,NaN,NaN,na,Gross Charges,752886.92,2
17235,1/20/2023,drg,17236,MS-DRG 003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,3,IP,NaN,NaN,na,Gross Charges,645099.41,3
17236,1/20/2023,drg,17237,MS-DRG 004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,4,IP,NaN,NaN,na,Gross Charges,949211.27,4
17237,1/20/2023,drg,17238,MS-DRG 005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,5,IP,NaN,NaN,na,Gross Charges,473562.45,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,na,Workers Comp Non Par,16561.95,987
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,na,Workers Comp Non Par,16775.05,988
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,na,Workers Comp Non Par,14799.61,989
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,na,Workers Comp Non Par,9671.26,998


In [14]:
df_mid_cdm = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'cdm'])
df_mid_cdm['code_prefix'] = 'none'
df_mid_cdm['code'] = 'na'
df_mid_cdm

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code_prefix,code


In [15]:
def code_orig_to_code_prefix(code_orig):
    if code_orig.startswith('APR-DRG'):
        return 'apr-drg'
    elif code_orig.startswith('MS-DRG'):
        return 'ms-drg'
    
    return 'none'

def pad_drg_if_needed(drg):
    if len(drg) == 1:
        return "00" + drg
    elif len(drg) == 2:
        return "0" + drg

    return drg

df_mid_drg['code_type'] = 'drg'
df_mid_drg['code_prefix'] = df_mid['code_orig'].apply(code_orig_to_code_prefix)
df_mid_drg['code'] = df_mid_drg['drg'].apply(pad_drg_if_needed)
df_mid_drg

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code,code_prefix
17233,1/20/2023,drg,17234,MS-DRG 001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1,IP,NaN,NaN,na,Gross Charges,846592.03,001,ms-drg
17234,1/20/2023,drg,17235,MS-DRG 002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,2,IP,NaN,NaN,na,Gross Charges,752886.92,002,ms-drg
17235,1/20/2023,drg,17236,MS-DRG 003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,3,IP,NaN,NaN,na,Gross Charges,645099.41,003,ms-drg
17236,1/20/2023,drg,17237,MS-DRG 004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,4,IP,NaN,NaN,na,Gross Charges,949211.27,004,ms-drg
17237,1/20/2023,drg,17238,MS-DRG 005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,5,IP,NaN,NaN,na,Gross Charges,473562.45,005,ms-drg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,na,Workers Comp Non Par,16561.95,987,ms-drg
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,na,Workers Comp Non Par,16775.05,988,ms-drg
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,na,Workers Comp Non Par,14799.61,989,ms-drg
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,na,Workers Comp Non Par,9671.26,998,ms-drg


In [16]:
df_mid = pd.concat([df_mid_cpt, df_mid_drg, df_mid_cdm])
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code,code_prefix
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,NaN,OP,C1751,,0272,Gross Charges,588.00,C1751,hcpcs_cpt
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,NaN,OP,A6537,,0272,Gross Charges,11.96,A6537,hcpcs_cpt
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,NaN,OP,C1729,,0272,Gross Charges,75.90,C1729,hcpcs_cpt
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,NaN,OP,C1894,,0272,Gross Charges,230.18,C1894,hcpcs_cpt
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,NaN,IP,A4648,,0278,Gross Charges,31.25,A4648,hcpcs_cpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,987,IP,NaN,NaN,na,Workers Comp Non Par,16561.95,987,ms-drg
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,988,IP,NaN,NaN,na,Workers Comp Non Par,16775.05,988,ms-drg
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,989,IP,NaN,NaN,na,Workers Comp Non Par,14799.61,989,ms-drg
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,998,IP,NaN,NaN,na,Workers Comp Non Par,9671.26,998,ms-drg


In [17]:
del df_mid['drg']
del df_mid['cpt']
df_mid['modifier'] = df_mid['modifier'].fillna('na')
df_mid['modifier'] = df_mid['modifier'].apply(lambda modifier: 'na' if len(modifier.strip()) == 0 else modifier) 
df_mid['rev_code'] = df_mid['rev_code'].fillna('na')

In [18]:
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,OP,na,0272,Gross Charges,588.00,C1751,hcpcs_cpt
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,OP,na,0272,Gross Charges,11.96,A6537,hcpcs_cpt
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,OP,na,0272,Gross Charges,75.90,C1729,hcpcs_cpt
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,OP,na,0272,Gross Charges,230.18,C1894,hcpcs_cpt
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,IP,na,0278,Gross Charges,31.25,A4648,hcpcs_cpt
...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,16561.95,987,ms-drg
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,16775.05,988,ms-drg
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,14799.61,989,ms-drg
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,IP,na,na,Workers Comp Non Par,9671.26,998,ms-drg


In [19]:
def payer_category_from_payer_orig(payer_orig):
    if payer_orig == "Gross Charges":
        return 'gross'
    elif payer_orig == "Discounted Cash Price":
        return 'cash'
    elif payer_orig == "De-Identified Minimum":
        return 'min'
    elif payer_orig == "De-Identified Maximum":
        return 'max'
    
    return 'payer'

df_mid['payer_category'] = df_mid['payer_orig'].apply(payer_category_from_payer_orig)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,OP,na,0272,Gross Charges,588.00,C1751,hcpcs_cpt,gross
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,OP,na,0272,Gross Charges,11.96,A6537,hcpcs_cpt,gross
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,OP,na,0272,Gross Charges,75.90,C1729,hcpcs_cpt,gross
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,OP,na,0272,Gross Charges,230.18,C1894,hcpcs_cpt,gross
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,IP,na,0278,Gross Charges,31.25,A4648,hcpcs_cpt,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,16561.95,987,ms-drg,payer
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,16775.05,988,ms-drg,payer
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,IP,na,na,Workers Comp Non Par,14799.61,989,ms-drg,payer
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,IP,na,na,Workers Comp Non Par,9671.26,998,ms-drg,payer


In [20]:
df_mid['patient_class'] = df_mid['patient_class'].replace("OP", "outpatient").replace("IP", "inpatient")
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,outpatient,na,0272,Gross Charges,588.00,C1751,hcpcs_cpt,gross
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,outpatient,na,0272,Gross Charges,11.96,A6537,hcpcs_cpt,gross
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,outpatient,na,0272,Gross Charges,75.90,C1729,hcpcs_cpt,gross
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,outpatient,na,0272,Gross Charges,230.18,C1894,hcpcs_cpt,gross
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,inpatient,na,0278,Gross Charges,31.25,A4648,hcpcs_cpt,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16561.95,987,ms-drg,payer
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16775.05,988,ms-drg,payer
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,14799.61,989,ms-drg,payer
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,inpatient,na,na,Workers Comp Non Par,9671.26,998,ms-drg,payer


In [21]:
df_mid['billing_class'] = 'na'

In [22]:
def plan_type_from_payer_orig(payer_orig):
    if "PPO" in payer_orig:
        return "PPO"
    elif "HMO" in payer_orig:
        return "HMO"
    
    return None

df_mid['plan_type'] = df_mid['payer_orig'].apply(plan_type_from_payer_orig)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category,billing_class,plan_type
0,1/20/2023,cpt,1,1000006S,3FR SL PROVENA MIDLINE MAX BAR,outpatient,na,0272,Gross Charges,588.00,C1751,hcpcs_cpt,gross,na,None
19,1/20/2023,cpt,20,1002031S,STOCKING ANTI THIGH MED REG,outpatient,na,0272,Gross Charges,11.96,A6537,hcpcs_cpt,gross,na,None
35,1/20/2023,cpt,36,1005203S,TRAY PARACENTESIS,outpatient,na,0272,Gross Charges,75.90,C1729,hcpcs_cpt,gross,na,None
38,1/20/2023,cpt,39,1006353S,SYSTEM ACCUSTICK II INTRODUCER,outpatient,na,0272,Gross Charges,230.18,C1894,hcpcs_cpt,gross,na,None
44,1/20/2023,cpt,45,1013233S,BRACHY GOLD SEED #SMG0242-025,inpatient,na,0278,Gross Charges,31.25,A4648,hcpcs_cpt,gross,na,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,1/20/2023,drg,18076,MS-DRG 987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16561.95,987,ms-drg,payer,na,None
1771836,1/20/2023,drg,18077,MS-DRG 988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16775.05,988,ms-drg,payer,na,None
1771837,1/20/2023,drg,18078,MS-DRG 989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,14799.61,989,ms-drg,payer,na,None
1771838,1/20/2023,drg,18079,MS-DRG 998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,inpatient,na,na,Workers Comp Non Par,9671.26,998,ms-drg,payer,na,None


In [23]:
df_mid['code_orig'] = None
df_mid['plan_orig'] = 'na'
df_mid['hospital_ccn'] = ccn
df_mid['hospital_ein'] = ein
df_mid['filename'] = filename

from dateutil.parser import parse as parse_datetime

df_mid['file_last_updated'] = df_mid['file_last_updated'].apply(
    lambda file_last_updated: parse_datetime(file_last_updated).isoformat().split("T")[0])

df_mid['url'] = url
df_mid['internal_code'] = df_mid['internal_code'].fillna('na')
df_mid['ndc'] = 'na'
df_mid['rate'] = df_mid['rate'].apply(lambda rate: None if type(rate) == str and rate.startswith("ERROR") else rate)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,...,code_prefix,payer_category,billing_class,plan_type,plan_orig,hospital_ccn,hospital_ein,filename,url,ndc
0,2023-01-20,cpt,1,None,3FR SL PROVENA MIDLINE MAX BAR,outpatient,na,0272,Gross Charges,588.00,...,hcpcs_cpt,gross,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
19,2023-01-20,cpt,20,None,STOCKING ANTI THIGH MED REG,outpatient,na,0272,Gross Charges,11.96,...,hcpcs_cpt,gross,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
35,2023-01-20,cpt,36,None,TRAY PARACENTESIS,outpatient,na,0272,Gross Charges,75.90,...,hcpcs_cpt,gross,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
38,2023-01-20,cpt,39,None,SYSTEM ACCUSTICK II INTRODUCER,outpatient,na,0272,Gross Charges,230.18,...,hcpcs_cpt,gross,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
44,2023-01-20,cpt,45,None,BRACHY GOLD SEED #SMG0242-025,inpatient,na,0278,Gross Charges,31.25,...,hcpcs_cpt,gross,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,2023-01-20,drg,18076,None,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16561.95,...,ms-drg,payer,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
1771836,2023-01-20,drg,18077,None,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,16775.05,...,ms-drg,payer,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
1771837,2023-01-20,drg,18078,None,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,na,na,Workers Comp Non Par,14799.61,...,ms-drg,payer,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na
1771838,2023-01-20,drg,18079,None,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,inpatient,na,na,Workers Comp Non Par,9671.26,...,ms-drg,payer,na,None,na,460041,68-0562507,680562507_DavisHospital_standardcharges.csv,https://content.steward.org/sites/default/file...,na


In [24]:
df_out = pd.DataFrame(columns=TARGET_COLUMNS)
df_out = df_out.append(df_mid)
df_out = df_out.dropna(subset=['rate'], axis=0)
df_out

/tmp/ipykernel_11616/1393279649.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_out = df_out.append(df_mid)


,hospital_ccn,hospital_ein,code_type,internal_code,description,code_orig,code_prefix,code,modifier,ndc,...,plan_name,plan_id,plan_type,rate,rate_method,rate_desc,filename,file_last_updated,url,permalink
0,460041,68-0562507,cpt,1,3FR SL PROVENA MIDLINE MAX BAR,None,hcpcs_cpt,C1751,na,na,...,NaN,NaN,NaN,588.00,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
19,460041,68-0562507,cpt,20,STOCKING ANTI THIGH MED REG,None,hcpcs_cpt,A6537,na,na,...,NaN,NaN,NaN,11.96,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
35,460041,68-0562507,cpt,36,TRAY PARACENTESIS,None,hcpcs_cpt,C1729,na,na,...,NaN,NaN,NaN,75.90,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
38,460041,68-0562507,cpt,39,SYSTEM ACCUSTICK II INTRODUCER,None,hcpcs_cpt,C1894,na,na,...,NaN,NaN,NaN,230.18,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
44,460041,68-0562507,cpt,45,BRACHY GOLD SEED #SMG0242-025,None,hcpcs_cpt,A4648,na,na,...,NaN,NaN,NaN,31.25,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771835,460041,68-0562507,drg,18076,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,None,ms-drg,987,na,na,...,NaN,NaN,NaN,16561.95,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
1771836,460041,68-0562507,drg,18077,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,None,ms-drg,988,na,na,...,NaN,NaN,NaN,16775.05,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
1771837,460041,68-0562507,drg,18078,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,None,ms-drg,989,na,na,...,NaN,NaN,NaN,14799.61,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN
1771838,460041,68-0562507,drg,18079,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,None,ms-drg,998,na,na,...,NaN,NaN,NaN,9671.26,NaN,NaN,680562507_DavisHospital_standardcharges.csv,2023-01-20,https://content.steward.org/sites/default/file...,NaN


In [25]:
df_out.to_csv(ccn + ".csv", index=False)